# Quantization Demo
This notebook demonstrates Int8 and Int4 quantization using NumPy.

In [ ]:
import numpy as np

def generate_random_matrix(rows: int, cols: int) -> np.ndarray:
    # Generates a random matrix with values uniformly distributed between -1 and 1
    return np.random.uniform(-1, 1, (rows, cols))

def calculate_mean_squared_error(original: np.ndarray, reconstructed: np.ndarray) -> float:
    # Calculates the Mean Squared Error between the original and reconstructed matrices
    return np.mean((original - reconstructed) ** 2)

### Int8 Quantization Explanation

The following code performs symmetric int8 quantization:

1.  **Calculate the Scale Factor**:
    ```python
    scale = np.max(np.abs(matrix)) / 127
    ```
    *   `np.abs(matrix)`: Takes the absolute value of every element.
    *   `np.max(...)`: Finds the largest absolute value.
    *   `/ 127`: Maps the range `[-max_abs, max_abs]` to `[-127, 127]`.

2.  **Quantize the Values**:
    ```python
    quantized = np.round(matrix / scale).astype(np.int8)
    ```
    *   `matrix / scale`: Normalizes data to the range -127 to 127.
    *   `np.round(...)`: Rounds to the nearest integer.
    *   `.astype(np.int8)`: Casts to 8-bit integer.

In [ ]:
def quantize_to_int8(matrix: np.ndarray) -> np.ndarray:
    # Quantizes the input matrix to int8
    scale = np.max(np.abs(matrix)) / 127  # Scale factor for int8 (-128 to 127, signed 8-bit integer)
    quantized = np.round(matrix / scale).astype(np.int8) # Ensure values fit in int8 range (round and cast)
    return quantized, scale

def dequantize_from_int8(quantized: np.ndarray, scale: float) -> np.ndarray:
    # Dequantizes the int8 matrix back to float32
    return quantized.astype(np.float32) * scale

### Int4 Quantization Explanation

The following code simulates 4-bit quantization (stored in int8):

1.  **Scale and Round**:
    ```python
    quantized = np.round(matrix / scale)
    ```
    *   **Scaling**: Divides by `scale` (calculated as `max_val / 7`) to map values to approx -7.0 to 7.0.
    *   **Rounding**: Converts to nearest whole number.

2.  **Clip and Cast**:
    ```python
    quantized = np.clip(quantized, -8, 7).astype(np.int8)
    ```
    *   **Clipping**: Forces values to stay within -8 and 7 (the range of a signed 4-bit integer).
    *   **Casting**: Stores the result in an 8-bit integer container (since NumPy lacks a native `int4` type).

In [ ]:
def quantize_to_int4(matrix: np.ndarray) -> np.ndarray:
    # Quantizes the input matrix to int4
    scale = np.max(np.abs(matrix)) / 7  # Scale factor for int4
    quantized = np.round(matrix / scale) #.astype(np.int8)???
    quantized = np.clip(quantized, -8, 7).astype(np.int8)  # Ensure values fit in int4 range
    return quantized, scale

def dequantize_from_int4(quantized: np.ndarray, scale: float) -> np.ndarray:
    # Dequantizes the int4 matrix back to float32
    return quantized.astype(np.float32) * scale

In [ ]:
np.random.seed(42)  # For reproducibility

rows, cols = 100, 100
original_matrix = generate_random_matrix(rows, cols)

# Int8 Quantization
quantized_int8, scale_int8 = quantize_to_int8(original_matrix)
reconstructed_int8 = dequantize_from_int8(quantized_int8, scale_int8)
mse_int8 = calculate_mean_squared_error(original_matrix, reconstructed_int8)
print(f"Int8 Quantization MSE: {mse_int8}")

# Int4 Quantization
quantized_int4, scale_int4 = quantize_to_int4(original_matrix)
reconstructed_int4 = dequantize_from_int4(quantized_int4, scale_int4)
mse_int4 = calculate_mean_squared_error(original_matrix, reconstructed_int4)
print(f"Int4 Quantization MSE: {mse_int4}")

print(f"Original Matrix Sample:\n{original_matrix}\n")
print(f"INT8 Quantized Sample:\n{quantized_int8}\n")
print(f"INT8 Dequantized Sample:\n{reconstructed_int8}\n")
print(f"INT8 MSE: {mse_int8}\n\n")
print(f"INT4 Quantized Sample:\n{quantized_int4}\n")
print(f"INT4 Dequantized Sample:\n{reconstructed_int4}\n")
print(f"INT4 MSE: {mse_int4}\n")